In [4]:
# h2o.shutdown()

In [5]:
import h2o
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/jupyter/.*')
h2o.init(nthreads = -1)


H2O cluster uptime:,56 minutes 57 seconds 62 milliseconds
H2O cluster version:,3.8.3.3
H2O cluster name:,H2O_started_from_python_laurend_cwt196
H2O cluster total nodes:,1
H2O cluster total free memory:,6.79 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [6]:
# A small clean telecommunications sample dataset (https://www.ibm.com/communities/analytics/watson-analytics-blog/predictive-insights-in-the-telco-customer-churn-data-set/
telco_dataset = h2o.import_file("https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv")


Parse Progress: [##################################################] 100%


In [7]:
# update the telco data format: 
# change SeniorCitizen to 'yes'/ 'no'
telco_dataset['SeniorCitizen'] = (telco_dataset['SeniorCitizen'] == 1).ifelse('Yes','No')
# Add the same form of capitalization across variables
# columns changed were: customerID, gender, tenure
telco_dataset.columns =[u'CustomerID',
 u'Gender',
 u'SeniorCitizen',
 u'Partner',
 u'Dependents',
 u'Tenure',
 u'PhoneService',
 u'MultipleLines',
 u'InternetService',
 u'OnlineSecurity',
 u'OnlineBackup',
 u'DeviceProtection',
 u'TechSupport',
 u'StreamingTV',
 u'StreamingMovies',
 u'Contract',
 u'PaperlessBilling',
 u'PaymentMethod',
 u'MonthlyCharges',
 u'TotalCharges',
 u'Churn']

In [8]:
# get a summary of the dataset
print telco_dataset.nacnt()
telco_dataset.describe()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0]
Rows:7,043 Cols:21

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,7,33.3333343,6.5 KB,2.8568119
C1N,1-Byte Integers (w/o NAs),11,52.3809552,76.4 KB,33.5680753
C2S,2-Byte Fractions,1,4.7619049,13.8 KB,6.0809709
C4S,4-Byte Fractions,1,4.7619049,27.6 KB,12.1258937
CStr,String,1,4.7619049,103.2 KB,45.3682482



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,227.6 KB,7043.0,1.0,21.0
mean,227.6 KB,7043.0,1.0,21.0
min,227.6 KB,7043.0,1.0,21.0
max,227.6 KB,7043.0,1.0,21.0
stddev,0 B,0.0,0.0,0.0
total,227.6 KB,7043.0,1.0,21.0


,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
type,string,enum,enum,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,enum
mins,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.25,18.8,0.0
mean,NaN,0.504756495811,0.162146812438,0.483032798523,0.299588243646,32.3711486582,0.903166264376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592219224762,NaN,64.7616924606,2283.30044084,0.265369870794
maxs,NaN,1.0,1.0,1.0,1.0,72.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,118.75,8684.8,1.0
sigma,NaN,0.500012873647,0.36861160561,0.49974751072,0.45811016751,24.5594810231,0.295752231784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.491456924049,NaN,30.0900470977,2266.77136188,0.441561305122
zeros,0,3488,5901,3641,4933,11,682,3390,2421,3498,3088,3095,3473,2810,2785,3875,2872,1544,0,0,5174
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
0,7590-VHVEG,Female,No,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,No,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [9]:
# what does each column tell you, make sure type is correct, and see if new features need to be created with strings
telco_dataset.columns

[u'CustomerID',
 u'Gender',
 u'SeniorCitizen',
 u'Partner',
 u'Dependents',
 u'Tenure',
 u'PhoneService',
 u'MultipleLines',
 u'InternetService',
 u'OnlineSecurity',
 u'OnlineBackup',
 u'DeviceProtection',
 u'TechSupport',
 u'StreamingTV',
 u'StreamingMovies',
 u'Contract',
 u'PaperlessBilling',
 u'PaymentMethod',
 u'MonthlyCharges',
 u'TotalCharges',
 u'Churn']

In [7]:
# check whether the customerID column is unique per row, if so use as an index and remove from the predictors
telco_dataset['CustomerID'].asfactor().unique().nrow


7043

In [8]:
# select all columns as predictors except the customerID (which is like an index) and the response column
features_list = list(telco_dataset.columns[1:-1])
response_name = 'Churn'

In [9]:
# specify the response column
response_col = telco_dataset[response_name]
# get a list of the categorical levels in your response column
print 'the response classes are:',response_col.levels()
print 'number of classes:', response_col.nlevels()
print ''
# check that the response column is already interpreted as a factor (i.e. enum/categorical)
print 'Is the response column a categorical:',response_col.isfactor()
# check that there are two levels in our response column:
response_col.nlevels()
print ''
# check for missing values in the training set and specifically the response column
print 'there are {0} missing values in the dataset'.format(telco_dataset.isna().sum())
print 'there are {0} missing values in the labels'.format(telco_dataset[response_col].isna().sum())
print ''

print 'check for class imbalace'
print '------------------------'
num_train_samples = telco_dataset.shape[0]  # Total number of training samples
print 'the dataset is not imbalanced, neither class is less then 10% of the whole (as shown below)'
telco_dataset[response_name].table()['Count']/num_train_samples



the response classes are: [['No', 'Yes']]
number of classes: [2]

Is the response column a categorical: [True]

there are 11.0 missing values in the dataset
there are 0.0 missing values in the labels

check for class imbalace
------------------------
the dataset is not imbalanced, neither class is less then 10% of the whole (as shown below)


Count
0.73463
0.26537


In [10]:
# create a new column that is the same as tenure but it replaces tenure = 0 with tenure = 1
# tenure = 0 corresponds to people who have be a customer less then a month but have still payed
# the monthly fee:
telco_dataset['new_tenure_col'] = (telco_dataset['Tenure'] == 0).ifelse(1, telco_dataset['Tenure'])

# impute TotalCharges from MonthlyCharges and tenure (i.e. TotalCharges = tenure * MonthlyCharges)
# if TotalCharges value is missing fill in, if not leave as is
telco_dataset['TotalCharges'] = (telco_dataset['TotalCharges'].isna() == 1).ifelse(
    (telco_dataset['MonthlyCharges']  * telco_dataset['new_tenure_col']), telco_dataset['TotalCharges'])

# remove new tenure col when done
telco_dataset= telco_dataset.drop('new_tenure_col')
# check that there are no more missing values
print telco_dataset['TotalCharges'].isna().sum()

0.0


In [11]:
# print out the count for each categorical feature level (this excludes monthly charges and total charges)
for column_name in telco_dataset.columns[1:-3]:
    print telco_dataset[str(column_name)].table()
    print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
    

Gender      Count
--------  -------
Female       3488
Male         3555

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  SeniorCitizen    Count
---------------  -------
              0     5901
              1     1142

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Partner      Count
---------  -------
No            3641
Yes           3402

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Dependents      Count
------------  -------
No               4933
Yes              2110

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  Tenure    Count
--------  -------
       0       11
       1      613
       2      238
       3      200
       4      176
       5      133
       6      110
       7      131
       8      123
       9      119

[73 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
PhoneService      Count
--------------  -------
No                  682
Yes                6361

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~

In [12]:
# split your datasets and set seed so the split is the same each time the code is run
train, valid, test = telco_dataset.split_frame(ratios=[0.70,0.15], seed=1234)

In [13]:
# check that the split went as expected, print out the dimensions of each dataset
# and then print their sum
print train.shape
print valid.shape
print test.shape
print train.shape[0] + valid.shape[0] + test.shape[0]

(4945, 21)
(1040, 21)
(1058, 21)
7043


In [14]:
# run gbm estimator with the default parameters to establish a benchmark model 

# import GBM estimator with default parameters (set seed for reproducibility)
from h2o.estimators.gbm import H2OGradientBoostingEstimator
default_model = H2OGradientBoostingEstimator(distribution= 'bernoulli', seed = 1234)

# train the default model
default_model.train(x=features_list, y=response_name, training_frame=train)

# get the AUC for the training set
print 'train auc:', default_model.auc() 

# get the AUC for the validation set
default_perf_on_valid = default_model.model_performance(valid)
print 'validation auc:', default_perf_on_valid.auc()


gbm Model Build Progress: [##################################################] 100%
train auc: 0.901181015724
validation auc: 0.848169251566


In [15]:
# using parameters from https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb
# this model overfits less on the training set
model_0 = H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    max_depth=4,
                                    learn_rate=0.01,
                                    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC",
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10)

model_0.train(x=features_list, 
              y=response_name, 
              training_frame=train,
              validation_frame = valid)

# get the AUC for the training set
print 'train auc:', model_0.auc() 

# get the AUC for the validation set
print 'validation auc:', model_0.auc(valid= True) 


gbm Model Build Progress: [##################################################] 100%
train auc: 0.87052692852
validation auc: 0.848343677777


## Grid Search
First we want to know what value of max_depth to use because it has a big impact on the model training time and optimal values depend strongly on the dataset. We'll do a quick Cartesian grid search to get a rough idea of good candidate max_depth values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.

In [16]:
# using grid to get the best max_depths
hyper_params = {'max_depth' : range(1,30,2)}
search_criteria = {'strategy': "Cartesian"}

# build gbm model with grid search parameters
from h2o.grid.grid_search import H2OGridSearch
gs_1 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.01,
                                    # learn_rate_annealing = 0.99, 
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10,              
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4),
                                    hyper_params = hyper_params,
                                    grid_id = 'grid_determines_max_depth',
                                    search_criteria = search_criteria)

# train grid search
gs_1.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)

# get the grid search results to see which max_depth performed the best
print(gs_1)


gbm Grid Build Progress: [##################################################] 100%
     max_depth                           model_ids              logloss
0            3   grid_determines_max_depth_model_1  0.41089112948814954
1            5   grid_determines_max_depth_model_2   0.4117401486993091
2            7   grid_determines_max_depth_model_3  0.42018792283642603
3            9   grid_determines_max_depth_model_4  0.42279651759726383
4           25  grid_determines_max_depth_model_12    0.427273130393641
5           19   grid_determines_max_depth_model_9  0.42742479030324043
6           11   grid_determines_max_depth_model_5  0.42784974174858514
7           13   grid_determines_max_depth_model_6  0.42792969707380263
8           15   grid_determines_max_depth_model_7  0.42816218737071216
9           17   grid_determines_max_depth_model_8   0.4281750472815081
10          29  grid_determines_max_depth_model_14   0.4283221055121523
11          21  grid_determines_max_depth_model_10  

In [17]:
# print out the auc for all models, sorted from best to worst
auc_table = gs_1.sort_by('auc(valid=True)',increasing=False)
print(auc_table)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,Hyperparameters: [max_depth],auc(valid=True)
grid_determines_max_depth_model_1,[3],0.8500031
grid_determines_max_depth_model_2,[5],0.8463959
grid_determines_max_depth_model_0,[1],0.8448988
grid_determines_max_depth_model_3,[7],0.8390458
grid_determines_max_depth_model_4,[9],0.8349952
grid_determines_max_depth_model_12,[25],0.8309471
grid_determines_max_depth_model_14,[29],0.8305256
grid_determines_max_depth_model_9,[19],0.8302494
grid_determines_max_depth_model_5,[11],0.8301549
grid_determines_max_depth_model_6,[13],0.8300653


In [18]:
# find the range of the max_depth for the top five models
new_auc_table = auc_table[1:5]
max_depths_to_use = new_auc_table['Hyperparameters: [max_depth]']
print max_depths_to_use

# get the max depths as a list
new_maxmin_list = []
for element in max_depths_to_use:
    new_maxmin_list.append(element[0])
new_max = max(new_maxmin_list)
new_min = min(new_maxmin_list)

([5], [1], [7], [9])


Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [19]:
hyper_params_2 = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_2 = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }


In [20]:
# printing out values in hyperparams_2 to give a sense of the range of values
for element in hyper_params_2.keys():
    print element
    print hyper_params_2[element]
    print "------------------------------"

col_sample_rate_change_per_level
[0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1]
------------------------------
nbins
[16, 32, 64, 128, 256, 512, 1024]
------------------------------
nbins_cats
[16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
------------------------------
col_sample_rate
[0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
------------------------------
sample_rate
[0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.3

In [21]:

#Build grid search with GBM and hyper parameters
gs_2 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    #learn_rate_annealing = 0.99,
                                    stopping_rounds = 5,
                                    stopping_tolerance = 1e-4,
                                    stopping_metric = "AUC", 
                                    score_tree_interval = 10,
                                    seed = 1234),
                                    hyper_params = hyper_params_2,
                                    grid_id = 'grid_2',
                                    search_criteria = search_criteria_2)

# train the grid and print results
gs_2.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)
print(gs_2)


gbm Grid Build Progress: [##################################################] 100%
     col_sample_rate col_sample_rate_change_per_level  \
0               0.44                             1.03   
1               0.73                              0.9   
2               0.35                             1.09   
3               0.71                              0.9   
4               0.91                             0.96   
5               0.45                              1.1   
6               0.96                             1.04   
7                0.4                             0.91   
8               0.81                             1.06   
9               0.27                             1.07   
10              0.67                             1.04   
11              0.52                              1.1   
12              0.46                             0.94   
13              0.33                             1.06   
14              0.87                              0.9   
15  

In [22]:
# print out the auc for all of the models on the validation set
auc_table_2 = gs_2.sort_by('auc(valid=True)',increasing=False)
print(auc_table_2)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [nbins, col_sample_rate, min_split_improvement, col_sample_rate_per_tree, min_rows, col_sample_rate_change_per_level, nbins_cats, sample_rate, histogram_type, max_depth]",auc(valid=True)
grid_2_model_9,"[256, 0.44, 1e-06, 0.38, 16.0, 1.03, 512, 0.55, u'QuantilesGlobal', 2]",0.8543565
grid_2_model_15,"[64, 0.4, 0.0, 0.34, 1024.0, 0.91, 2048, 0.48, u'UniformAdaptive', 7]",0.8532082
grid_2_model_38,"[64, 0.35, 1e-08, 0.83, 32.0, 1.09, 128, 0.69, u'QuantilesGlobal', 1]",0.8529030
grid_2_model_45,"[1024, 0.73, 0.0001, 0.6, 64.0, 0.9, 256, 0.29, u'QuantilesGlobal', 2]",0.8527940
grid_2_model_41,"[32, 0.91, 0.0001, 0.4, 32.0, 0.96, 1024, 0.67, u'RoundRobin', 2]",0.8525759
---,---,---
grid_2_model_26,"[32, 0.77, 1e-06, 0.75, 16.0, 1.06, 32, 0.29, u'QuantilesGlobal', 6]",0.8384280
grid_2_model_8,"[16, 0.57, 0.0, 0.68, 4.0, 1.1, 4096, 0.58, u'RoundRobin', 8]",0.8361145
grid_2_model_16,"[16, 0.42, 1e-06, 0.45, 1.0, 1.08, 256, 0.35, u'RoundRobin', 9]",0.8340092
grid_2_model_14,"[128, 0.5, 0.0, 0.92, 2.0, 0.94, 2048, 0.61, u'RoundRobin', 9]",0.8334545



See the whole table with table.as_data_frame()



In [37]:
# get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model('grid_2_model_9')
test_performance_model = best_model.model_performance(test)

# get the performance on the test model
print test_performance_model.auc()

0.861834373852


In [38]:
# save a csv of the predictions:
# first get the predictions from the best model
best_model_predictions = best_model.predict(test)
best_model_predictions.head()


gbm prediction Progress: [##################################################] 100%


predict,No,Yes
No,0.951522,0.0484778
No,0.964498,0.0355024
Yes,0.593113,0.406887
No,0.846744,0.153256
Yes,0.504991,0.495009
Yes,0.234364,0.765636
Yes,0.49124,0.50876
Yes,0.71928,0.28072
No,0.966616,0.0333842
Yes,0.433083,0.566917


In [39]:
# export predictions as a csv to the current directory
# h2o.export_file(best_model_predictions,'best_model_predictions.csv')


Export File Progress: [##################################################] 100%


In [42]:
# download a POJO of the best model (best_model), you have to specify the path to 
# where you want your pojo saved (not provided below)
# otherwise it will print to screen
# h2o.download_pojo(best_model)